# Maillage 1D

Pour calculer numériquement la solution d'une EDP par la méthode des différences finies, il faut commencer par construire le maillage, c'est-à-dire l'ensemble des points où la solution approchée sera calculée.

Dans ce notebook, nous allons construire une classe `mesh1D` qui nous sera utile tout au long de ce cours. Nous la plaçons donc dans le fichier `mesh1D.hpp` afin de pouvoir l'utiliser dans d'autres notebooks facilement.

In [1]:
#include <iostream>
#include <vector>
#include <map>

## Les paramètres

Lorsque l'on fait des simulations numériques, de nombreux paramètres sont utilisés et il peut être souvent pratique d'avoir une structure permettant de les stocker.

### Question 1

Complétez la cellule suivante afin de mettre en paramètres 
* `xmin` la borne inférieur de l'intervalle de simulation
* `xmax` la borne supérieur de l'intervalle de simulation
* `Nx` le nombre de points du maillage (a priori un entier)

In [2]:
using parameters = std::map<std::string, double>;

parameters p{
    {"xmin", 0},
    {"xmax", 1},
    {"Nx", 10}
};

### Question 2

Affichez les paramètres "joliement" !

In [3]:
std::cout << std::string(80, '-') << std::endl;
std::cout << "parameters" << std::endl;
std::cout << std::string(80, '-') << std::endl;
for (const auto & pk : p)
    std::cout << pk.first << ": \t" << pk.second << std::endl;
std::cout << std::string(80, '-') << std::endl;

--------------------------------------------------------------------------------
parameters
--------------------------------------------------------------------------------
Nx: 	10
xmax: 	1
xmin: 	0
--------------------------------------------------------------------------------


## Une première classe

Nous allons compléter la classe `mesh1D` ainsi que ses deux classes filles `mesh1D_cc` (cell-centered) et `mesh1D_nc` (node-centered) afin qu'elles répondent à nos attentes.

### Question 3

Complétez les constructeurs qui ne sont pas par défaut (ceux qui utilisent les paramètres) afin d'initialiser les maillages :
* pour le maillage _cell-centered_, les points sont au centre des cellules (et ne touchent donc pas le bord),
* pour le maillage _node-centered_, les points sont au bord des cellules.

### Question 4

Complétez la fonction d'affichage afin de pouvoir visualiser les maillages (attention n'affichez peut-être pas tous les points du maillage pour éviter que ce ne soit trop long).

In [4]:
%%file mesh1D.hpp

using parameters = std::map<std::string, double>;

/*****************************************************************************************
*
* class mesh1D to define the mesh of the simulation in 1D
*
*****************************************************************************************/
class mesh1D
{
public:
    double xmin, xmax, dx; // left, right bounds + spatial step
    std::size_t Nx;        // number of points
    std::vector<double> x; // the points of the mesh
    std::string name;      // the name of the class
    
    mesh1D() = default;                    // empty constructor
    mesh1D(const mesh1D&) = default;         // copy constructor
    mesh1D& operator=(mesh1D&& m) = default; // move constructor
    mesh1D(parameters);
};

mesh1D::mesh1D(parameters param)
{
    name = "generic";
    xmin = param["xmin"];
    xmax = param["xmax"];
    Nx = param["Nx"];
    if (Nx <= 0)
    {
        std::cout << "Error in mesh generation: the parameter 'Nx' has to be positive" << std::endl;
        Nx = 1;
    }
    dx = 0.;
    x.resize(Nx);
}

// print
std::ostream& operator<<(std::ostream& out, mesh1D& m)
{
    out << std::string(80, '-') << std::endl;
    out << "mesh informations (" << m.name << ")" << std::endl;
    out << std::string(80, '-') << std::endl;
    // BEGIN SOLUTION
    out << "    dimension:         " << 1      << std::endl;
    out << "    xmin:              " << m.xmin << std::endl;
    out << "    xmax:              " << m.xmax << std::endl;
    out << "    number of points:  " << m.Nx   << std::endl;
    out << "    spatial step:      " << m.dx   << std::endl;
    out << "[" << std::endl;
    if (m.Nx <= 6)
        for (auto &xk : m.x)
            out << "  " << xk << std::endl;
    else
    {
        for (auto it = m.x.begin(); it != std::next(m.x.begin(),3); ++it)
            out << "  " << *it << std::endl;
        out << "   ... " << std::endl;
        for (auto it = std::prev(m.x.end(), 3); it != m.x.end(); ++it)
            out << "  " << *it << std::endl;
    }
    out << "]" << std::endl;
    // END SOLUTION
    out << std::string(80, '-') << std::endl;
    return out;
}

class mesh1D_cc: public mesh1D
{
public:
    mesh1D_cc() = default;
    mesh1D_cc(parameters);
};

// constructor
mesh1D_cc::mesh1D_cc(parameters param): mesh1D(param)
{
    name = "cell-centered";
    // BEGIN SOLUTION
    dx = (xmax - xmin) / Nx;
    x[0] = xmin + 0.5*dx;
    for (auto k = 1; k < x.size(); ++k)
        x[k] = x[k-1] + dx;
    // END SOLUTION
}

class mesh1D_nc: public mesh1D
{
public:
    mesh1D_nc() = default;
    mesh1D_nc(parameters);
};

// constructor
mesh1D_nc::mesh1D_nc(parameters param): mesh1D(param)
{
    name = "node-centered";
    // BEGIN SOLUTION
    dx = (xmax - xmin) / (Nx - 1);
    x[0] = xmin;
    for (auto k = 1; k < x.size(); ++k)
        x[k] = x[k-1] + dx;
    // END SOLUTION
}



Overwriting mesh1D.hpp


### Question 5

Exécutez la cellule suivante pour vérifier que tout fonctionne correctement. 

In [5]:
#include "mesh1D.hpp"

In [6]:
mesh1D_cc mc(p);
std::cout << mc;

--------------------------------------------------------------------------------
mesh informations (cell-centered)
--------------------------------------------------------------------------------
    dimension:         1
    xmin:              0
    xmax:              1
    number of points:  10
    spatial step:      0.1
[
  0.05
  0.15
  0.25
   ... 
  0.75
  0.85
  0.95
]
--------------------------------------------------------------------------------


In [7]:
mesh1D_nc mn(p);
std::cout << mn;

--------------------------------------------------------------------------------
mesh informations (node-centered)
--------------------------------------------------------------------------------
    dimension:         1
    xmin:              0
    xmax:              1
    number of points:  10
    spatial step:      0.111111
[
  0
  0.111111
  0.222222
   ... 
  0.777778
  0.888889
  1
]
--------------------------------------------------------------------------------


Voici un petit script utilisant `xplot` afin de visualiser les points du maillage dans une fenêtre graphique.

In [8]:
#include "xplot/xfigure.hpp"
#include "xplot/xmarks.hpp"
#include "xplot/xaxes.hpp"

In [9]:
std::vector<mesh1D> list_mesh = {mc, mn};
std::vector<std::string> list_color = {"orange", "purple"};
auto n = list_mesh.size();

xpl::linear_scale sx, sy;
auto sx_min = 1e13, sx_max = -1.e13;
for (auto& m: list_mesh)
{
    if (m.xmin < sx_min)
        sx_min = m.xmin;
    if (m.xmax > sx_max)
        sx_max = m.xmax;
}
sx.min = sx_min, sx.max = sx_max;
sy.min = 0, sy.max = n;

auto ax_x = xpl::axis_generator(sx)
    .label("x")
    .finalize();
auto ax_y = xpl::axis_generator(sy)
    .visible(false)
    .orientation("vertical")
    .side("left")
    .finalize();

std::vector<xpl::scatter> list_scatter(n);
std::vector<xpl::lines> list_lines(n);
std::vector<double> list_labels_x(n, 0.);
std::vector<double> list_labels_y(n);
std::vector<std::string> list_name(n);

for (auto k = 0; k<n; ++k)
{
    list_scatter[k] = xpl::scatter_generator(sx, sy)
        .x(list_mesh[k].x)
        .y(std::vector<double> (list_mesh[k].Nx, k))
        .colors(std::vector<xtl::xoptional<std::string>>{list_color[k]})
        .marker("circle")
        .finalize();
    list_lines[k] = xpl::lines_generator(sx, sy)
        .x(std::vector<double> {list_mesh[k].xmin, list_mesh[k].xmax})
        .y(std::vector<double> (2, k))
        .finalize();
    list_labels_y[k] = .25 + k;
    list_name[k] = list_mesh[k].name;
}
auto labels = xpl::label_generator(sx, sy)
    .x(list_labels_x)
    .y(list_labels_y)
    .text(list_name)
    .colors(list_color)
    .font_weight("bolder")
    .default_size(26)
    .finalize();
auto fig = xpl::figure_generator()
    .padding_x(0.025)
    .padding_y(0.025)
    .title("Mesh with " + std::to_string((int)p["Nx"]) + " points")
    .finalize();

for (auto k = 0; k<n; ++k)
{
    fig.add_mark(list_lines[k]);
    fig.add_mark(list_scatter[k]);
}
fig.add_mark(labels);
fig.add_axis(ax_x);
fig.add_axis(ax_y);
fig.display()

A Jupyter widget